In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from qick import *
from qick.helpers import gauss
from tqdm import tqdm_notebook as tqdm

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
sys.path.append('C:\\_Lib\\python\\rfsoc\\rfsoc_multimode\\example_expts')
# sys.path.append('C:\\_Lib\\python\\multimode')
import scipy as sp
import json
from scipy.fft import fft, fftfreq

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, get_current_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})

Warning serial library import failed.
Could not load InstrumentManagerWindow
Could not import QickInstrument, try install qick package
Could not load AD5780 dac 
Could not load labbrick
Could not load Autonics TM4 Driver


In [19]:
import json
import os
# path = os.getcwd()
path = r'H:\Shared drives\SLab\Multimode\experiment\240911' # for data 
print("path: ", path)

path:  H:\Shared drives\SLab\Multimode\experiment\240911


# setup experiment hardware

In [20]:
expt_path=path+'\\data1'

print('Data will be stored in', expt_path)

Data will be stored in H:\Shared drives\SLab\Multimode\experiment\240911\data1


In [24]:
curr_path = os.getcwd()
config_file = curr_path+'\\config_rfsoc_ziqian.yml'
config_path = config_file
print('Config will be', config_path)

exp_param_file = curr_path+'\\experiment_config.yml'
exp_param_path = exp_param_file
print('Exp Param will be', exp_param_path)

Config will be c:\_Lib\python\multimode_expts\config_rfsoc_ziqian.yml
Exp Param will be c:\_Lib\python\multimode_expts\experiment_config.yml


In [25]:
qubit_i = 0
import yaml
with open(config_file, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [26]:

im = InstrumentManager(ns_address='192.168.137.25') # SLAC lab

In [27]:
im.keys()

dict_keys(['Pyro.NameServer', 'Qick101'])

In [28]:
print(im['Qick101'])

<Pyro4.core.Proxy at 0x23c488cbf40; not connected; for PYRO:obj_838aa0c048e94c2480cf8174087be298@192.168.137.101:38869>


In [29]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)


QICK configuration:

	Board: ZCU216

	Software version: 0.2.180
	Firmware timestamp: Tue Oct 15 12:06:51 2024

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - tProc output 0, envelope memory 32768 samples
		DAC tile 2, blk 0, 32-bit DDS, fabric=399.360 MHz, f_dds=6389.760 MHz
	1:	axis_signal_gen_v6 - tProc output 1, envelope memory 32768 samples
		DAC tile 2, blk 1, 32-bit DDS, fabric=399.360 MHz, f_dds=6389.760 MHz
	2:	axis_signal_gen_v6 - tProc output 2, envelope memory 32768 samples
		DAC tile 2, blk 2, 32-bit DDS, fabric=399.360 MHz, f_dds=6389.760 MHz
	3:	axis_signal_gen_v6 - tProc output 3, envelope memory 32768 samples
		DAC tile 2, blk 3, 32-bit DDS, fabric=399.360 MHz, f_dds=6389.760 MHz
	4:	axis_signal_gen_v6 - tProc output 4, envelope memory 32768 samples
		DAC tile 3, blk 0, 32-bit DDS, fabric=430.080 MHz, f_dds=6881.280 MHz
	5:	axis_signal_gen_v6 - tProc output 5, envelope memory 32768 samples
		DAC ti

# choose experiment name

In [30]:
import sys

# Path to add
expts_path = 'C:\\_Lib\\python\\multimode_expts'

# Add the path to the system path at the highest priority
if expts_path not in sys.path:
    sys.path.insert(0, expts_path)
    print('Path added at highest priority')

# Verify the path is added
print(sys.path)

# Import the experiments module from multimode
import experiments as meas

# Verify the module is imported from the correct path
print(meas.__file__)

['C:\\_Lib\\python\\multimode_expts', 'c:\\_Lib\\python\\multimode_expts', 'C:\\_Lib\\python', 'C:\\_Lib\\python\\slab', 'C:\\_Lib\\python\\qick\\qick_lib', 'C:\\_Lib\\python\\rfsoc\\rfsoc_multimode\\example_expts', 'c:\\_Lib\\python\\multimode_expts', 'c:\\Users\\26049\\.conda\\envs\\slab\\python38.zip', 'c:\\Users\\26049\\.conda\\envs\\slab\\DLLs', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib', 'c:\\Users\\26049\\.conda\\envs\\slab', '', 'C:\\Users\\26049\\AppData\\Roaming\\Python\\Python38\\site-packages', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\easy_lmfit-1.0-py3.8.egg', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\attrdict-2.0.1-py3.8.egg', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\win32', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\win32\\lib', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\Pythonwin', '/home/xilinx/jupyter_notebooks/', 'C:\\_Lib\

# Run experiments

In [43]:
experiment_class, experiment_name = 'single_qubit.t1', 'T1Experiment'
experiment_class, experiment_name = 'single_qubit.single_shot', 'HistogramExperiment'
# experiment_class, experiment_name = 'single_qubit.single_shot_old', 'HistogramExperiment_oldold'
# experiment_class, experiment_name = 'single_qubit.single_shot', 'HistogramExperiment_old'
# experiment_class, experiment_name = 'qubit_cavity.non_demolution_readout', 'DemolutionMeasurementExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_ramsey', 'RamseyExperiment'
# experiment_class, experiment_name = 'single_qubit.phase_sweep', 'PhaseSweepExperiment'
# experiment_class, experiment_name = 'single_qubit.phase_sweep_averager', 'PhaseSweepAveragerExperiment'
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi', 'AmplitudeRabiExperiment'
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi_general', 'AmplitudeRabiGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi_ef', 'AmplitudeRabiEFExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_echo', 'RamseyEchoExperiment'
# experiment_class, experiment_name = 'single_qubit.cavity_spectroscopy', 'CavitySpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.resonator_spectroscopy', 'ResonatorSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.length_rabi_general', 'LengthRabiGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.length_rabi_f0g1_general', 'LengthRabiGeneralF0g1Experiment'
# experiment_class, experiment_name = 'qubit_cavity.single_mode_wigner_tomography', 'WignerTomography1ModeExperiment'
# experiment_class, experiment_name = 'qubit_cavity.single_mode_cats_wigner_tomography', 'WignerTomography1ModeCatExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_spectroscopy', 'PulseProbeSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_f0g1_spectroscopy', 'PulseProbeF0g1SpectroscopyExperiment' # <--------------
# experiment_class, experiment_name = 'qubit_cavity.ecd', 'ECDExperiment'
# experiment_class, experiment_name = 'qubit_cavity.ecd_read_spec', 'ECDReadSpecExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_ef_spectroscopy', 'PulseProbeEFSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.t1_cavity', 'T1CavityExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_cavity', 'CavityRamseyExperiment' # -------
# experiment_class, experiment_name = 'single_qubit.t2_cavity', 'CavityRamseyExperiment_old'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_spectroscopy_prepulse', 'PulseProbePrePulseSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.cavity2qubit', 'Cavity2QubitSweepExperiment'
# experiment_class, experiment_name = 'single_qubit.sideband_general', 'SidebandGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.rf_flux_spectroscopy', 'FluxSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.parity_delay', 'ParityDelayExperiment'
# experiment_class, experiment_name = 'single_qubit.parity_gain', 'ParityGainExperiment'
# experiment_class, experiment_name = 'single_qubit.parity_freq', 'ParityFreqExperiment'
# experiment_class, experiment_name = 'single_qubit.single_shot_prepulse','HistogramPrepulseExperiment'
# experiment_class, experiment_name = 'single_qubit.rb_ziqian', 'SingleRB'
# experiment_class, experiment_name = 'single_qubit.rbam', 'MultiRBAM'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate', 'SingleBeamSplitterRB'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_postselection', 'SingleBeamSplitterRBPostSelection'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_check_target', 'SingleBeamSplitterRB_check_target'
experiment_class, experiment_name = 'single_qubit.rb_BSgate_ramsey', 'SingleBeamSplitterRB_ramsey'
# experiment_class, experiment_name = 'qubit_cavity.displacement_enhanced_sideband', 'DisplacementEnhancedSidebandExperiment'
# experiment_class, experiment_name = 'single_qubit.sideband_t1_general', 'SidebandT1GeneralExperiment'
# experiment_class, experiment_name = 'qubit_cavity.out_and_back', 'OutAndBackExperiment'
# experiment_class, experiment_name = 'qubit_cavity.amplitude_rabi_f0g1', 'AmplitudeRabiF0g1Experiment'
# experiment_class, experiment_name = 'single_qubit.rf_flux_spectroscopy_f0g1', 'FluxSpectroscopyF0g1Experiment' 
# experiment_class, experiment_name = 'single_qubit.dragpulse_calibration', 'DragCalibrationExperiment' 
# experiment_class, experiment_name = 'single_qubit.fast_reset_test', 'FastResetExperiment'
# experiment_class, experiment_name = 'single_qubit.active_reset_single_shot', 'HistogramActiveResetExperiment'

# 

run_exp = eval(f"meas.{experiment_class}.{experiment_name}(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path)")


with open(exp_param_file, 'r') as file:

    # Load the YAML contenta
    loaded = yaml.safe_load(file)
    run_exp.cfg.expt = eval(f"loaded['{experiment_name}']")

# special updates on device_config file
# run_exp.cfg.device.readout.relax_delay =
#  [1500] # Wait time between experiments [us]
run_exp.cfg.device.readout.relax_delay = [100] # best for active reset
# run_exp.cfg.device.readout.relax_delay = [2500] # Wait time between experiments [us]
# run_exp.cfg.device.manipulate.readout_length = 5
# run_exp.cfg.device.storage.readout_length = 5
#run_exp.cfg.device.readout.relax_delay = int(qick.c)
print(run_exp.cfg)
run_exp.go(analyze=False, display=False, progress=True, save=True)

{'prepulse_qubit': {'freq': [3568.1751638611518], 'length': [0.06], 'gain': [30], 'phase': [0], 'repeat_time': 10}, 'device': {'qubit': {'T1': [127.0], 'f_ge': [3568.1751638611518], 'f_ge_resolved': [3568.1751638611518], 'f_ge_idle': [3568.1751638611518], 'f_ef': [3425.5564066067], 'f_ef_idle': [3425.5564066067], 'ramp_sigma': [0.005], 'ramp_sigma_num': [3], 'pulses': {'pi_ge': {'gain': [10716], 'sigma': [0.035], 'length': [0], 'type': 'gauss'}, 'pi_ge_ftop': {'gain': [3000], 'sigma': [0.005], 'length': [0.190914187445986], 'type': 'flat_top'}, 'pi_ge_fast': {'gain': [10682], 'sigma': [0.035], 'length': [0], 'type': 'gauss'}, 'pi_ge_resolved': {'gain': [50], 'sigma': [0.005], 'length': [6.553697146578866], 'type': 'flat_top', 'repeat_no': 1}, 'hpi_ge': {'gain': [5510], 'length': [0], 'sigma': [0.035], 'type': 'gauss'}, 'hpi_ge_ftop': {'gain': [3000], 'sigma': [0.005], 'length': [0.09131395179924079], 'type': 'flat_top'}, 'hpi_ge_fast': {'gain': [5195], 'length': [0], 'sigma': [0.035], 

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

697
706
ge fidelity (%): 27.576298259233223
rotation angle (deg): 51.12625284001639
threshold ge: -100.34246991019165


  0%|          | 0/40 [00:00<?, ?it/s]

running list: [[6, 4], [4, 4, 4, 5, 4, 6, 6, 2], [1, 4, 3, 2, 5, 1, 5, 2, 3, 5, 6, 3, 5, 2, 3, 5], [5, 2, 1, 1, 6, 5, 5, 1, 6, 4, 6, 3, 6, 5, 4, 1, 3, 2, 5, 2, 1, 6, 6], [3, 1, 3, 2, 1, 3, 1, 3, 3, 5, 3, 4, 2, 4, 2, 2, 6, 4, 4, 5, 2, 6, 6, 1, 5, 4, 1, 1, 1, 3], [3, 6, 1, 3, 3, 3, 2, 5, 6, 4, 3, 4, 3, 3, 5, 1, 5, 4, 3, 2, 4, 1, 3, 2, 3, 2, 6, 4, 4, 4, 6, 3, 5, 2, 4, 5], [5, 4, 2, 3, 3, 6, 4, 1, 6, 4, 5, 5, 4, 3, 1, 1, 5, 2, 5, 2, 2, 5, 5, 2, 6, 5, 6, 2, 4, 2, 1, 3, 5, 3, 1, 4, 1, 5, 5, 3, 6, 6, 2, 5], [1, 1, 4, 1, 1, 4, 6, 3, 5, 6, 6, 3, 1, 3, 5, 6, 4, 2, 2, 6, 5, 2, 5, 2, 2, 6, 2, 5, 3, 6, 6, 6, 5, 4, 4, 1, 3, 2, 5, 3, 4, 2, 5, 1, 6, 3, 4, 5, 1, 4, 4], [4, 6, 2, 3, 2, 6, 2, 3, 2, 6, 1, 1, 3, 4, 1, 1, 5, 2, 3, 3, 2, 1, 2, 5, 5, 5, 1, 5, 2, 4, 6, 5, 6, 4, 2, 1, 5, 6, 3, 5, 3, 3, 1, 2, 6, 6, 1, 6, 4, 3, 5, 4, 6, 1, 1, 6, 6, 3], [6, 2, 6, 5, 4, 6, 6, 4, 3, 2, 1, 3, 2, 4, 1, 5, 1, 4, 3, 1, 1, 1, 1, 2, 6, 1, 6, 3, 1, 4, 2, 6, 3, 4, 5, 2, 2, 4, 2, 5, 4, 1, 5, 3, 5, 6, 1, 6, 4, 1, 5, 1, 1, 5, 

KeyboardInterrupt: 

In [45]:
arr = [[6, 4], [4, 4, 4, 5, 4, 6, 6, 2], [1, 4, 3, 2, 5, 1, 5, 2, 3, 5, 6, 3, 5, 2, 3, 5], [5, 2, 1, 1, 6, 5, 5, 1, 6, 4, 6, 3, 6, 5, 4, 1, 3, 2, 5, 2, 1, 6, 6], [3, 1, 3, 2, 1, 3, 1, 3, 3, 5, 3, 4, 2, 4, 2, 2, 6, 4, 4, 5, 2, 6, 6, 1, 5, 4, 1, 1, 1, 3], [3, 6, 1, 3, 3, 3, 2, 5, 6, 4, 3, 4, 3, 3, 5, 1, 5, 4, 3, 2, 4, 1, 3, 2, 3, 2, 6, 4, 4, 4, 6, 3, 5, 2, 4, 5], [5, 4, 2, 3, 3, 6, 4, 1, 6, 4, 5, 5, 4, 3, 1, 1, 5, 2, 5, 2, 2, 5, 5, 2, 6, 5, 6, 2, 4, 2, 1, 3, 5, 3, 1, 4, 1, 5, 5, 3, 6, 6, 2, 5], [1, 1, 4, 1, 1, 4, 6, 3, 5, 6, 6, 3, 1, 3, 5, 6, 4, 2, 2, 6, 5, 2, 5, 2, 2, 6, 2, 5, 3, 6, 6, 6, 5, 4, 4, 1, 3, 2, 5, 3, 4, 2, 5, 1, 6, 3, 4, 5, 1, 4, 4], [4, 6, 2, 3, 2, 6, 2, 3, 2, 6, 1, 1, 3, 4, 1, 1, 5, 2, 3, 3, 2, 1, 2, 5, 5, 5, 1, 5, 2, 4, 6, 5, 6, 4, 2, 1, 5, 6, 3, 5, 3, 3, 1, 2, 6, 6, 1, 6, 4, 3, 5, 4, 6, 1, 1, 6, 6, 3], [6, 2, 6, 5, 4, 6, 6, 4, 3, 2, 1, 3, 2, 4, 1, 5, 1, 4, 3, 1, 1, 1, 1, 2, 6, 1, 6, 3, 1, 4, 2, 6, 3, 4, 5, 2, 2, 4, 2, 5, 4, 1, 5, 3, 5, 6, 1, 6, 4, 1, 5, 1, 1, 5, 5, 5, 5, 5, 6, 4, 4, 1, 3, 1, 4], [6, 5, 6, 2, 3, 6, 4, 4, 1, 2, 4, 1, 4, 1, 3, 1, 2, 5, 6, 6, 4, 6, 3, 1, 4, 6, 4, 3, 5, 5, 6, 4, 4, 2, 1, 6, 3, 4, 6, 1, 5, 5, 2, 6, 3, 6, 2, 1, 1, 6, 5, 6, 4, 6, 2, 5, 2, 4, 6, 6, 3, 6, 2, 1, 2, 6, 3, 5, 2, 6, 3, 1], [4, 2, 6, 4, 1, 3, 1, 3, 5, 3, 5, 2, 1, 1, 3, 4, 3, 1, 5, 5, 2, 4, 6, 5, 3, 3, 1, 4, 3, 2, 4, 5, 5, 3, 2, 5, 2, 3, 1, 5, 5, 2, 1, 5, 1, 5, 1, 3, 3, 6, 6, 1, 2, 2, 1, 1, 6, 2, 2, 6, 1, 4, 6, 5, 5, 6, 1, 3, 5, 6, 1, 6, 3, 6, 1, 3, 6, 3, 3], [1, 4, 5, 2, 1, 1, 2, 5, 3, 3, 3, 3, 3, 1, 6, 1, 3, 2, 3, 5, 6, 3, 5, 3, 4, 5, 3, 2, 1, 4, 6, 2, 2, 2, 1, 6, 1, 4, 5, 4, 1, 3, 2, 3, 2, 5, 5, 5, 5, 6, 3, 1, 5, 1, 6, 3, 5, 1, 4, 3, 1, 2, 1, 6, 4, 3, 3, 1, 3, 5, 4, 4, 2, 5, 1, 5, 6, 1, 3, 6, 3, 3, 1, 6, 2, 5], [5, 3, 5, 2, 5, 2, 6, 1, 3, 5, 3, 5, 1, 1, 3, 3, 2, 4, 1, 3, 5, 5, 2, 6, 3, 5, 6, 2, 5, 5, 3, 3, 2, 2, 1, 1, 5, 1, 5, 5, 2, 6, 4, 3, 1, 1, 5, 2, 4, 4, 1, 1, 6, 3, 4, 2, 2, 6, 5, 1, 2, 6, 4, 1, 4, 2, 1, 4, 5, 6, 1, 5, 1, 3, 5, 1, 6, 2, 2, 6, 1, 4, 4, 3, 4, 5, 2, 2, 4, 6, 1, 6, 1], [6, 1, 2, 4, 1, 4, 6, 6, 3, 4, 4, 5, 1, 4, 1, 2, 1, 3, 4, 1, 3, 1, 2, 1, 4, 1, 4, 6, 2, 2, 5, 3, 1, 3, 3, 2, 4, 6, 1, 1, 4, 1, 5, 2, 1, 6, 2, 6, 3, 3, 4, 3, 4, 4, 5, 4, 1, 3, 3, 4, 5, 2, 3, 6, 6, 1, 1, 2, 6, 4, 6, 6, 5, 6, 2, 5, 2, 3, 3, 1, 1, 1, 3, 4, 1, 5, 2, 1, 2, 2, 4, 2, 2, 2, 6, 4, 1, 2, 5, 1], [4, 5, 4, 5, 3, 5, 1, 5, 6, 4, 2, 6, 4, 3, 3, 6, 4, 2, 2, 6, 5, 1, 1, 4, 6, 1, 4, 1, 3, 5, 3, 1, 1, 5, 3, 6, 5, 5, 2, 1, 2, 3, 1, 4, 5, 3, 4, 3, 5, 4, 4, 2, 5, 3, 2, 1, 2, 1, 4, 2, 5, 1, 1, 3, 1, 6, 1, 4, 5, 6, 3, 3, 5, 6, 6, 5, 4, 2, 4, 3, 5, 2, 3, 1, 5, 3, 1, 4, 5, 4, 6, 1, 4, 1, 6, 1, 1, 6, 6, 4, 3, 5, 4, 6, 1, 4, 1], [4, 4, 6, 1, 1, 2, 3, 1, 6, 1, 2, 2, 5, 4, 5, 2, 5, 3, 2, 3, 1, 3, 2, 4, 6, 4, 6, 4, 3, 3, 6, 4, 6, 3, 4, 1, 3, 6, 4, 3, 5, 4, 6, 5, 1, 4, 1, 2, 2, 3, 2, 1, 6, 2, 1, 3, 5, 2, 3, 2, 3, 2, 1, 1, 5, 6, 5, 1, 2, 6, 4, 1, 3, 2, 2, 6, 6, 2, 6, 6, 5, 5, 2, 3, 1, 3, 2, 5, 2, 2, 3, 3, 6, 2, 3, 1, 2, 6, 5, 3, 3, 6, 5, 6, 6, 3, 2, 6, 2, 5, 3, 6, 5, 3], [2, 6, 4, 3, 6, 4, 2, 4, 2, 5, 5, 1, 3, 1, 5, 2, 3, 5, 5, 5, 2, 3, 1, 4, 5, 1, 1, 3, 6, 1, 1, 6, 2, 6, 5, 6, 1, 6, 3, 3, 4, 3, 6, 5, 1, 3, 5, 6, 5, 6, 6, 5, 5, 6, 2, 6, 1, 1, 2, 1, 5, 3, 1, 2, 3, 2, 1, 6, 2, 6, 6, 6, 4, 3, 1, 4, 6, 5, 4, 1, 1, 6, 6, 3, 6, 1, 1, 3, 4, 2, 1, 3, 6, 5, 2, 4, 4, 2, 1, 5, 6, 1, 5, 1, 2, 2, 1, 4, 2, 6, 2, 6, 5, 5, 5, 6, 1, 2, 1, 1, 5], [5, 4, 2, 4, 2, 1, 4, 5, 2, 3, 5, 2, 5, 1, 4, 3, 6, 4, 2, 2, 2, 6, 6, 3, 5, 4, 1, 2, 6, 2, 6, 6, 5, 6, 4, 4, 2, 3, 2, 2, 3, 4, 4, 5, 3, 6, 1, 4, 6, 3, 5, 2, 3, 4, 1, 4, 3, 5, 6, 1, 5, 2, 3, 1, 6, 3, 3, 6, 1, 6, 5, 3, 2, 4, 1, 4, 1, 6, 2, 1, 2, 4, 1, 5, 6, 1, 2, 3, 1, 5, 4, 6, 5, 5, 1, 1, 2, 4, 6, 3, 3, 1, 2, 6, 4, 6, 2, 1, 2, 1, 2, 3, 1, 4, 2, 3, 5, 2, 5, 3, 5, 3, 2, 5, 3, 6, 4, 6], [4, 2, 6, 6, 4, 2, 3, 1, 4, 4, 6, 1, 6, 5, 6, 2, 5, 2, 2, 1, 3, 2, 2, 2, 1, 3, 2, 3, 6, 4, 1, 1, 6, 6, 2, 1, 1, 6, 2, 6, 2, 4, 6, 4, 4, 2, 4, 4, 5, 4, 1, 4, 1, 6, 6, 2, 3, 5, 6, 1, 2, 5, 3, 1, 5, 3, 6, 6, 6, 3, 5, 1, 2, 3, 1, 1, 5, 4, 2, 1, 5, 6, 5, 2, 5, 2, 2, 5, 2, 1, 5, 4, 4, 4, 4, 4, 2, 1, 6, 5, 4, 3, 2, 4, 6, 2, 4, 1, 2, 5, 4, 5, 4, 6, 2, 1, 5, 4, 3, 4, 4, 4, 5, 5, 2, 1, 5, 5, 2, 5, 5, 2, 5, 3, 6], [6, 6, 4, 2, 2, 5, 2, 6, 3, 1, 1, 2, 5, 2, 1, 2, 4, 2, 4, 2, 2, 6, 1, 1, 5, 6, 5, 4, 5, 6, 4, 1, 4, 2, 4, 1, 1, 3, 2, 4, 6, 2, 1, 5, 3, 2, 5, 6, 4, 2, 4, 6, 2, 2, 2, 5, 5, 4, 1, 3, 3, 2, 4, 1, 3, 3, 6, 5, 3, 3, 1, 3, 3, 2, 2, 4, 6, 2, 1, 1, 2, 6, 6, 4, 1, 1, 2, 1, 6, 5, 5, 6, 5, 5, 2, 5, 2, 4, 2, 6, 1, 1, 5, 5, 1, 4, 1, 6, 6, 2, 4, 4, 5, 2, 1, 2, 3, 1, 6, 2, 4, 5, 2, 3, 1, 4, 6, 4, 4, 1, 5, 4, 5, 3, 6, 6, 2, 3, 2, 2, 5, 4], [5, 4, 3, 3, 2, 4, 4, 5, 1, 4, 2, 3, 2, 1, 3, 5, 2, 3, 4, 6, 4, 3, 6, 1, 1, 5, 5, 6, 4, 1, 1, 4, 1, 1, 1, 6, 4, 3, 5, 6, 2, 4, 1, 4, 3, 5, 6, 1, 2, 2, 4, 2, 1, 3, 6, 1, 3, 5, 1, 4, 6, 6, 5, 2, 3, 4, 5, 2, 3, 2, 2, 5, 3, 2, 3, 4, 2, 6, 2, 4, 5, 1, 1, 2, 4, 2, 3, 6, 5, 1, 6, 2, 5, 5, 6, 1, 5, 5, 4, 4, 5, 2, 4, 6, 6, 5, 6, 1, 2, 2, 3, 5, 5, 2, 5, 4, 3, 4, 3, 3, 6, 2, 4, 1, 3, 5, 1, 1, 2, 1, 1, 2, 6, 1, 3, 5, 4, 5, 2, 6, 2, 3, 6, 6, 2, 3, 6, 2, 4], [3, 2, 1, 5, 5, 2, 6, 2, 2, 1, 2, 1, 2, 5, 5, 1, 4, 5, 6, 4, 6, 4, 5, 4, 2, 6, 5, 1, 3, 6, 1, 1, 4, 5, 2, 5, 2, 6, 5, 4, 3, 4, 1, 6, 2, 6, 4, 6, 2, 3, 2, 1, 5, 2, 5, 5, 5, 3, 3, 6, 1, 3, 4, 4, 3, 6, 4, 5, 2, 6, 2, 3, 4, 6, 2, 6, 6, 3, 2, 1, 5, 4, 2, 3, 4, 3, 4, 4, 2, 5, 6, 2, 1, 4, 4, 3, 6, 2, 6, 1, 6, 1, 2, 5, 5, 5, 6, 1, 5, 5, 3, 5, 1, 5, 5, 1, 5, 3, 6, 6, 4, 2, 5, 2, 3, 4, 1, 5, 5, 6, 1, 5, 4, 3, 6, 1, 5, 6, 3, 6, 3, 6, 4, 4, 6, 4, 3, 3, 6, 6, 1, 4, 6, 2, 2, 3], [4, 6, 6, 5, 2, 2, 6, 3, 6, 3, 4, 3, 6, 2, 1, 4, 6, 4, 1, 5, 5, 2, 1, 1, 3, 2, 4, 6, 4, 2, 5, 6, 2, 4, 4, 3, 2, 2, 4, 1, 2, 6, 4, 5, 1, 3, 1, 4, 5, 2, 6, 3, 6, 3, 3, 6, 5, 6, 4, 3, 4, 5, 1, 3, 5, 1, 2, 1, 5, 3, 2, 5, 3, 3, 5, 5, 4, 2, 3, 3, 6, 3, 5, 1, 1, 1, 2, 5, 4, 5, 5, 2, 4, 5, 5, 3, 4, 6, 5, 6, 4, 6, 2, 5, 3, 3, 6, 2, 3, 2, 4, 6, 1, 5, 4, 5, 5, 3, 5, 2, 5, 4, 6, 5, 1, 3, 2, 6, 5, 3, 3, 3, 6, 3, 2, 3, 3, 5, 1, 6, 6, 3, 1, 2, 3, 6, 4, 5, 3, 1, 1, 5, 6, 6, 6, 2, 4, 2, 2, 3, 3, 4, 3], [5, 3, 5, 4, 5, 4, 4, 4, 4, 1, 4, 6, 4, 3, 5, 6, 5, 6, 4, 4, 6, 5, 5, 1, 5, 6, 1, 6, 6, 2, 6, 6, 5, 6, 3, 6, 2, 5, 6, 4, 5, 5, 2, 2, 5, 5, 5, 1, 4, 3, 6, 4, 1, 3, 5, 4, 2, 5, 1, 1, 6, 2, 2, 1, 6, 5, 6, 1, 3, 6, 1, 2, 5, 6, 6, 5, 4, 3, 5, 6, 4, 1, 3, 4, 4, 6, 6, 6, 5, 6, 3, 3, 5, 5, 6, 1, 2, 5, 3, 5, 1, 6, 2, 4, 4, 4, 1, 6, 1, 4, 1, 4, 3, 1, 5, 3, 2, 4, 4, 6, 4, 4, 3, 5, 5, 6, 1, 2, 4, 1, 4, 2, 1, 3, 1, 2, 2, 2, 4, 5, 3, 1, 1, 2, 2, 6, 2, 4, 5, 3, 2, 1, 3, 1, 6, 6, 3, 2, 6, 5, 2, 1, 1, 6, 2, 4, 6, 5, 3], [4, 5, 2, 1, 4, 2, 1, 1, 1, 4, 3, 1, 5, 6, 2, 3, 2, 2, 2, 4, 2, 2, 4, 3, 1, 6, 6, 4, 6, 2, 5, 3, 1, 6, 6, 6, 3, 3, 6, 3, 6, 3, 1, 5, 3, 3, 1, 3, 2, 5, 4, 2, 2, 1, 5, 3, 2, 2, 4, 2, 6, 4, 5, 2, 4, 5, 6, 6, 1, 5, 5, 6, 2, 4, 4, 1, 3, 1, 5, 3, 1, 3, 6, 5, 2, 6, 4, 1, 1, 2, 3, 5, 6, 4, 3, 2, 4, 4, 2, 2, 6, 4, 5, 3, 2, 1, 1, 1, 2, 6, 1, 5, 5, 2, 2, 1, 6, 2, 1, 1, 5, 2, 2, 1, 1, 3, 1, 6, 1, 6, 6, 6, 5, 2, 6, 5, 4, 5, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 5, 5, 5, 4, 1, 6, 4, 2, 5, 4, 4, 2, 2, 1, 6, 1, 6, 1, 5, 5, 2, 5, 1, 2, 5], [1, 2, 2, 1, 6, 5, 6, 2, 5, 6, 6, 5, 2, 5, 6, 2, 5, 5, 4, 6, 3, 4, 3, 4, 1, 5, 5, 6, 4, 6, 2, 3, 5, 4, 5, 2, 1, 5, 2, 3, 2, 1, 2, 4, 2, 5, 3, 1, 1, 3, 4, 4, 2, 6, 5, 2, 1, 3, 1, 2, 4, 3, 6, 2, 1, 4, 4, 1, 5, 2, 3, 6, 2, 4, 2, 3, 4, 3, 6, 1, 1, 2, 1, 3, 1, 2, 2, 2, 3, 1, 6, 4, 5, 3, 2, 4, 5, 1, 6, 5, 2, 6, 5, 6, 2, 3, 5, 6, 1, 5, 4, 2, 3, 4, 1, 6, 4, 3, 5, 4, 6, 6, 2, 1, 1, 5, 4, 1, 6, 1, 4, 5, 4, 4, 2, 4, 4, 2, 5, 4, 3, 4, 3, 1, 1, 1, 1, 1, 6, 5, 3, 6, 2, 2, 2, 3, 1, 3, 4, 4, 3, 1, 2, 2, 4, 5, 2, 2, 5, 6, 2, 2, 5, 4, 2, 1, 1, 3, 4, 5, 5, 2, 3], [4, 6, 1, 2, 5, 2, 5, 3, 4, 1, 3, 6, 6, 5, 1, 4, 6, 5, 5, 2, 4, 6, 3, 3, 5, 1, 5, 4, 1, 6, 2, 2, 2, 4, 1, 4, 3, 1, 3, 2, 6, 5, 2, 3, 2, 5, 6, 2, 3, 5, 4, 3, 5, 6, 4, 5, 1, 2, 4, 5, 5, 6, 3, 3, 6, 1, 6, 5, 1, 4, 2, 4, 2, 3, 1, 2, 5, 5, 5, 4, 6, 6, 3, 3, 6, 5, 5, 3, 6, 3, 3, 2, 6, 1, 6, 3, 1, 6, 1, 5, 5, 2, 2, 1, 2, 1, 6, 6, 3, 6, 4, 6, 3, 4, 2, 4, 1, 5, 1, 1, 5, 1, 1, 5, 2, 2, 1, 6, 2, 1, 3, 5, 3, 2, 2, 3, 3, 6, 4, 1, 2, 3, 4, 4, 5, 1, 2, 1, 3, 2, 6, 1, 1, 2, 2, 2, 1, 5, 6, 2, 2, 3, 3, 2, 5, 6, 6, 6, 2, 3, 4, 6, 4, 2, 4, 4, 2, 2, 1, 6, 2, 2, 2, 6, 6, 1, 4, 6, 2, 3, 6], [1, 4, 5, 1, 5, 2, 2, 2, 3, 3, 4, 1, 5, 3, 2, 4, 2, 5, 6, 6, 1, 4, 1, 3, 6, 1, 1, 4, 1, 6, 4, 2, 3, 1, 1, 3, 6, 1, 5, 2, 5, 4, 2, 1, 3, 1, 4, 4, 4, 2, 6, 5, 4, 2, 4, 1, 1, 6, 5, 3, 4, 5, 2, 6, 4, 2, 3, 5, 3, 6, 4, 4, 2, 1, 2, 2, 5, 2, 2, 6, 2, 2, 2, 3, 6, 4, 3, 2, 2, 5, 6, 2, 3, 2, 1, 4, 4, 1, 5, 4, 5, 6, 2, 1, 2, 4, 3, 3, 6, 5, 4, 1, 2, 4, 5, 4, 3, 5, 2, 6, 4, 6, 3, 2, 2, 6, 2, 4, 1, 5, 1, 4, 2, 6, 5, 6, 3, 4, 3, 5, 6, 1, 2, 4, 2, 1, 1, 4, 3, 2, 1, 2, 4, 1, 5, 2, 4, 3, 4, 4, 1, 3, 6, 3, 4, 5, 1, 1, 3, 6, 1, 2, 2, 3, 4, 2, 3, 3, 5, 2, 5, 2, 6, 1, 3, 2, 1, 4, 3, 1, 1, 2, 5, 2, 2, 2, 5, 1], [1, 6, 3, 1, 6, 3, 1, 1, 4, 4, 2, 1, 5, 4, 1, 4, 3, 4, 6, 5, 3, 2, 6, 1, 6, 3, 4, 6, 6, 1, 2, 4, 3, 1, 2, 4, 2, 6, 4, 3, 6, 5, 6, 3, 1, 1, 1, 3, 5, 6, 6, 5, 2, 4, 3, 5, 4, 3, 3, 1, 4, 4, 1, 2, 6, 6, 5, 5, 2, 1, 1, 1, 3, 5, 1, 3, 5, 3, 6, 2, 5, 1, 6, 6, 2, 5, 6, 1, 6, 6, 2, 3, 4, 6, 5, 5, 2, 4, 2, 3, 2, 2, 2, 6, 6, 3, 3, 1, 1, 4, 2, 5, 5, 2, 1, 2, 4, 6, 3, 3, 6, 5, 2, 3, 4, 3, 4, 4, 2, 6, 2, 1, 4, 6, 3, 5, 6, 4, 3, 6, 5, 3, 4, 3, 3, 1, 6, 2, 4, 4, 3, 3, 3, 6, 3, 4, 3, 1, 2, 3, 3, 5, 1, 5, 6, 1, 3, 5, 2, 3, 4, 3, 3, 3, 3, 6, 1, 5, 1, 3, 4, 1, 4, 1, 6, 6, 1, 1, 6, 1, 2, 3, 5, 3, 3, 6, 3, 6, 4, 4, 2, 2, 1, 3, 5]]
len(arr[-1])

205

In [13]:
from MM_dual_rail_base import *

In [20]:
mm_base = MM_dual_rail_base(yaml_cfg)
mm_base.get_total_time_from_running_list(running_list = [0, 3], bs_time = 1)

1

In [96]:
depth_list = [1, 5, 10, 20, 30, 40,
               50, 75, 100, 125, 150,
                 175, 200, 250, 300, 400,
                   500, 750, 1000]
reps_list = [1000, 1000, 1000, 1000, 2000, 2000,
             3000, 3000, 3000, 3000, 3000,
             3000, 3000, 3000, 5000, 5000,
             5000, 10000, 10000]

len(depth_list), len(reps_list)


(19, 19)

In [30]:
from multimode_expts.MM_dual_rail_base import  *
dummy = MM_dual_rail_base( cfg=yaml_cfg)
dummy.prepulse_str_for_random_ram_state(1, [1], 6, 6)

[['qubit', 'ge', 'hpi', -90],
 ['qubit', 'ef', 'pi', 0],
 ['man', 'M1', 'pi', 0],
 ['storage', 'M1-S6', 'pi', 0]]

In [ ]:
t = 0.01
cycles2us = soc.cycles2us(1)
int(round(t / cycles2us))

In [ ]:
Ec = 3425.5537944060666 - 3568.1751638611518
Ej = (Ec+3568.1751638611518)**2/8/Ec
print(Ej, Ec, Ej/Ec)

m = 2
epsilon = Ec*2**(4*m+5)/math.factorial(m)*np.sqrt(2/np.pi)*(Ej/2/Ec)**(m/2+3/4)*np.exp(-np.sqrt(8*Ej/Ec))*1e3

print(epsilon, 'kHz')
print('Ramsey limit: ', 1000/epsilon/2/np.pi, 'us')

$\text{Gain (linear)} = 10^{\frac{\text{dB}}{20}} $

# Run sequential experiment

In [11]:
import multimode_expts.sequential_experiment as seq_exp

experiment_name = 'length_rabi_sweep'
# experiment_name = 'dc_flux_sweep'
experiment_name = 'length_rabi_f0g1_sweep'
# experiment_name = 'gain_displace_sweep'
experiment_name = 'sideband_general_sweep'
# experiment_name = 'single_shot_phase_sweep'
# experiment_name = 'single_shot_freq_sweep'
# experiment_name = 'HistogramExperiment_freq_sweep'
# experiment_name = 'HistogramExperiment_ef_freq_sweep'
# experiment_name = 'single_shot_time_sweep'
# experiment_name = 'single_qubit_tomography'      # <--------------
# experiment_name = 'SingleRB_sweep_freq'
# experiment_name = 'SingleRB_sweep_pi_amp'
# experiment_name = 'SingleRB_sweep_hpi_amp'
# experiment_name = 'SingleRB_sweep_f0g1_phase'
# experiment_name = 'SingleRB_sweep_depth_and_prepulse'
# experiment_name = 'SingleRB_sweep_depth'
# experiment_name = 'MultiRBAM_sweep_depth'
# experiment_name = 'SingleBeamsplitterRB_sweep_depth'
# experiment_name = 'SingleBeamSplitterRB_sweep_freq'
# experiment_name = 'SingleBeamSplitterRB_gain_freq'
# experiment_name = 'pulseprobe_f0g1_dc_flux_sweep'
# experiment_name = 'PhaseSweepAverager_sweep_reps_middlepulse'
# experiment_name = 'cavity_t1_dc_flux_sweep'
# experiment_name = 'displace_enhanced_sweep'
# experiment_name = 'fluxspectroscopy_f0g1_dc_flux_sweep'
# experiment_name = 'storage_sideband_sweep'
# experiment_name = 'sideband_fidelity_optimization'
# experiment_name = 'storage_t1_optimization'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth'#
experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_and_ram'
experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_and_single_spec'
experiment_name = 'DualRail_sweep_depth_and_single_spec_and_stor'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_storsweep'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_defined_storsweep'
experiment_class, experiment_name = 'MM_dual_rail_seq_exp', 'SingleBeamSplitterRB_check_target_sweep_depth'
# experiment_name = 'ramsey_sweep'
# experiment_name = 'cavity_ramsey_sweep'
# experiment_name = 'manipulate_dc_flux_sweep'
# experiment_name = 'rb_bs_optimization'
# experiment_name = 'rb_bs_dual_rail_optimization'
# experiment_name = 'cross_kerr_sweep'
# experiment_name = 'single_dual_rail_tomography'
# experiment_name = 'two_dual_rail_tomography'




class_for_exp = eval('seq_exp.' + experiment_class)()
eval('class_for_exp .' + experiment_name)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)
# eval('seq_exp.' + experiment_name)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

Config will be h:\Shared drives\SLab\Multimode\experiment\240911\config_rfsoc_ziqian.yml
Index: 0 depth. = 1 
{'pre_sweep_pulse': [['qubit', 'ge', 'hpi', 0], ['qubit', 'ef_new', 'pi', 0], ['man', 'M1', 'pi', 0], ['storage', 'M1-S1', 'pi', 0]], 'post_sweep_pulse': [['storage', 'M1-S1', 'pi', 0], ['man', 'M1', 'pi', 0], ['qubit', 'ef_new', 'pi', 0], ['qubit', 'ge', 'hpi', 0]], 'wait_freq': 0.671, 'prepulse': True, 'postpulse': True, 'rb_time': 'None', 'bs_gate_num': 'None', 'qubits': [0], 'singleshot_reps': 5000, 'span': 500, 'active_reset': True, 'man_reset': True, 'storage_reset': True, 'threshold': -39.769721213292996, 'readout_per_round': 4, 'prep_man_photon': True, 'rb_active_reset': True, 'rb_man_reset': True, 'rb_storage_reset': True, 'rb_reps': 80, 'gates_per_wait': 100000, 'rounds': 1, 'variations': 60, 'rb_depth': 1, 'IRB_gate_no': -1, 'bs_repeat': 1, 'sync': False, 'setup': False, 'bs_para': [520.925, 5000, 0.565451994946885, 0.005]}
sscfg active reset True
4951
4917
ge fideli

  0%|          | 0/60 [00:00<?, ?it/s]

Saving h:\Shared drives\SLab\Multimode\experiment\240911\data\00293_SingleBeamSplitterRB_check_target_sweep_depth.h5
Index: 1 depth. = 6 
{'pre_sweep_pulse': [['qubit', 'ge', 'hpi', 0], ['qubit', 'ef_new', 'pi', 0], ['man', 'M1', 'pi', 0], ['storage', 'M1-S1', 'pi', 0]], 'post_sweep_pulse': [['storage', 'M1-S1', 'pi', 0], ['man', 'M1', 'pi', 0], ['qubit', 'ef_new', 'pi', 0], ['qubit', 'ge', 'hpi', 546.3623355974782]], 'wait_freq': 0.671, 'prepulse': True, 'postpulse': True, 'rb_time': 'None', 'bs_gate_num': 'None', 'qubits': [0], 'singleshot_reps': 5000, 'span': 500, 'active_reset': True, 'man_reset': True, 'storage_reset': True, 'threshold': -39.769721213292996, 'readout_per_round': 4, 'prep_man_photon': True, 'rb_active_reset': True, 'rb_man_reset': True, 'rb_storage_reset': True, 'rb_reps': 480, 'gates_per_wait': 100000, 'rounds': 1, 'variations': 60, 'rb_depth': 6, 'IRB_gate_no': -1, 'bs_repeat': 1, 'sync': False, 'setup': False, 'bs_para': [520.925, 5000, 0.565451994946885, 0.005]

  0%|          | 0/60 [00:00<?, ?it/s]

Saving h:\Shared drives\SLab\Multimode\experiment\240911\data\00294_SingleBeamSplitterRB_check_target_sweep_depth.h5
Index: 2 depth. = 11 
{'pre_sweep_pulse': [['qubit', 'ge', 'hpi', 0], ['qubit', 'ef_new', 'pi', 0], ['man', 'M1', 'pi', 0], ['storage', 'M1-S1', 'pi', 0]], 'post_sweep_pulse': [['storage', 'M1-S1', 'pi', 0], ['man', 'M1', 'pi', 0], ['qubit', 'ef_new', 'pi', 0], ['qubit', 'ge', 'hpi', 1092.7246711949567]], 'wait_freq': 0.671, 'prepulse': True, 'postpulse': True, 'rb_time': 'None', 'bs_gate_num': 'None', 'qubits': [0], 'singleshot_reps': 5000, 'span': 500, 'active_reset': True, 'man_reset': True, 'storage_reset': True, 'threshold': -39.769721213292996, 'readout_per_round': 4, 'prep_man_photon': True, 'rb_active_reset': True, 'rb_man_reset': True, 'rb_storage_reset': True, 'rb_reps': 880, 'gates_per_wait': 100000, 'rounds': 1, 'variations': 60, 'rb_depth': 11, 'IRB_gate_no': -1, 'bs_repeat': 1, 'sync': False, 'setup': False, 'bs_para': [520.925, 5000, 0.565451994946885, 0.0

...

Saving h:\Shared drives\SLab\Multimode\experiment\240911\data\00370_SingleBeamSplitterRB_check_target_sweep_depth.h5
Index: 78 depth. = 391 
{'pre_sweep_pulse': [['qubit', 'ge', 'hpi', 0], ['qubit', 'ef_new', 'pi', 0], ['man', 'M1', 'pi', 0], ['storage', 'M1-S1', 'pi', 0]], 'post_sweep_pulse': [['storage', 'M1-S1', 'pi', 0], ['man', 'M1', 'pi', 0], ['qubit', 'ef_new', 'pi', 0], ['qubit', 'ge', 'hpi', 71163.69421157196]], 'wait_freq': 0.671, 'prepulse': True, 'postpulse': True, 'rb_time': 'None', 'bs_gate_num': 'None', 'qubits': [0], 'singleshot_reps': 5000, 'span': 500, 'active_reset': True, 'man_reset': True, 'storage_reset': True, 'threshold': -39.769721213292996, 'readout_per_round': 4, 'prep_man_photon': True, 'rb_active_reset': True, 'rb_man_reset': True, 'rb_storage_reset': True, 'rb_reps': 31280, 'gates_per_wait': 100000, 'rounds': 1, 'variations': 60, 'rb_depth': 391, 'IRB_gate_no': -1, 'bs_repeat': 1, 'sync': False, 'setup': False, 'bs_para': [520.925, 5000, 0.565451994946885,

  0%|          | 0/60 [00:00<?, ?it/s]

Saving h:\Shared drives\SLab\Multimode\experiment\240911\data\00371_SingleBeamSplitterRB_check_target_sweep_depth.h5
Index: 79 depth. = 396 
{'pre_sweep_pulse': [['qubit', 'ge', 'hpi', 0], ['qubit', 'ef_new', 'pi', 0], ['man', 'M1', 'pi', 0], ['storage', 'M1-S1', 'pi', 0]], 'post_sweep_pulse': [['storage', 'M1-S1', 'pi', 0], ['man', 'M1', 'pi', 0], ['qubit', 'ef_new', 'pi', 0], ['qubit', 'ge', 'hpi', 70344.15070817579]], 'wait_freq': 0.671, 'prepulse': True, 'postpulse': True, 'rb_time': 'None', 'bs_gate_num': 'None', 'qubits': [0], 'singleshot_reps': 5000, 'span': 500, 'active_reset': True, 'man_reset': True, 'storage_reset': True, 'threshold': -39.769721213292996, 'readout_per_round': 4, 'prep_man_photon': True, 'rb_active_reset': True, 'rb_man_reset': True, 'rb_storage_reset': True, 'rb_reps': 31680, 'gates_per_wait': 100000, 'rounds': 1, 'variations': 60, 'rb_depth': 396, 'IRB_gate_no': -1, 'bs_repeat': 1, 'sync': False, 'setup': False, 'bs_para': [520.925, 5000, 0.565451994946885,

  0%|          | 0/60 [00:00<?, ?it/s]

Saving h:\Shared drives\SLab\Multimode\experiment\240911\data\00372_SingleBeamSplitterRB_check_target_sweep_depth.h5


In [58]:
from multimode_expts.MM_rb_base import  *
def get_parity_str(man_mode_no, return_pulse=False): 
    '''
    Create parity pulse 
    '''
    parity_str = [['qubit', 'ge', 'hpi', 0],
                  ['qubit', 'ge', 'parity_M' + str(man_mode_no), 0],
                  ['qubit', 'ge', 'hpi', 0]]
    if return_pulse:
        mm_base = MM_rb_base(cfg = yaml_cfg)
        creator = mm_base.get_prepulse_creator(parity_str)
        return creator.pulse.tolist()

    return parity_str


In [ ]:
from MM_base import MM_base
stor_no=1
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['man', 'M' + str(man_idx) , 'pi',0],
                       ['qubit', 'ef', 'pi',0]]
creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


In [ ]:
from MM_base import MM_base
stor_no=1
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no), 'pi',0],
    ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


In [36]:
from MM_base import MM_base
stor_no=[1,2]
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0]]

creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


[[3568.1751638611518, 3425.5564066067, 2007.06, 349.415, 520.925, 520.925],
 [10716, 8236, 15000, 3000, 5000, 5000],
 [0, 0, 0.651287, 0.4885714285714285, 0.565451994946885, 0.565451994946885],
 [0, 0, 0, 0, 0, 0],
 [2, 2, 0, 1, 1, 1],
 ['g', 'g', 'flat_top', 'flat_top', 'flat_top', 'flat_top'],
 [0.035, 0.035, 0.005, 0.005, 0.005, 0.005]]

In [43]:
from MM_base import MM_base
stor_no=[[1,2],[3,4]]
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][1]), 'hpi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][1]), 'hpi',0],
                    ['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][1]), 'hpi',0]]

creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1

[[3568.1751638611518,
  3425.5564066067,
  2007.06,
  349.415,
  520.925,
  520.925,
  3568.1751638611518,
  3425.5564066067,
  2007.06,
  696.665,
  875.845,
  875.845],
 [10716, 8236, 15000, 3000, 5000, 5000, 10716, 8236, 15000, 9000, 9000, 9000],
 [0,
  0,
  0.651287,
  0.4885714285714285,
  0.565451994946885,
  0.565451994946885,
  0,
  0,
  0.651287,
  0.666955,
  0.6271437312608766,
  0.6271437312608766],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 1],
 ['g',
  'g',
  'flat_top',
  'flat_top',
  'flat_top',
  'flat_top',
  'g',
  'g',
  'flat_top',
  'flat_top',
  'flat_top',
  'flat_top'],
 [0.035,
  0.035,
  0.005,
  0.005,
  0.005,
  0.005,
  0.035,
  0.035,
  0.005,
  0.005,
  0.005,
  0.005]]

In [29]:
stor_no=[3,4]
man_idx=1
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]# measure]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
I_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]# measure]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
I_gate2 = creator.pulse.tolist()


I_gate = [I_gate1, I_gate2]

post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
X_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
X_gate2 = creator.pulse.tolist()

X_gate = [X_gate1, X_gate2]

post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',90], 
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
Y_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
Y_gate2 = creator.pulse.tolist()

Y_gate = [Y_gate1, Y_gate2]

# Prepulse and Postpulse generator

In [21]:
freq = 1423.04
pi_length =0.8425040638970613*2
pi2_length = 0.8425040638970613
gain = 13500

dataset.update_all('M1-S7', freq, 0.01, pi_length, pi2_length, gain)

## Dataset

In [14]:
from dataset import * 
from dataset import storage_man_swap_dataset

In [15]:
dataset = storage_man_swap_dataset('man1_storage_swap_dataset.csv')
dataset.df

,stor_name,freq (MHz),precision (MHz),pi (mus),h_pi (mus),gain (DAC units),last_update
0,M1-S1,349.415,0.01,0.977143,0.488571,3000,2024-10-26 17:42:46.915401
1,M1-S2,520.925,0.01,1.130904,0.565452,5000,2024-10-26 17:43:09.376354
2,M1-S3,696.665,0.01,1.333910,0.666955,9000,2024-10-26 17:43:34.019332
3,M1-S4,875.845,0.01,1.254287,0.627144,9000,2024-10-26 17:43:51.867832
4,M1-S5,1053.395,0.01,1.973198,0.986599,7500,2024-10-26 17:44:14.975970
5,M1-S6,1245.160,0.01,1.595310,0.797655,13500,2024-10-26 17:44:34.143253
6,M1-S7,1423.040,0.01,1.685008,0.842504,13500,2024-10-26 17:44:51.587301
7,M1-S8,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3
8,M1-S9,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3
9,M1-S10,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3


In [ ]:
list(dataset.df['gain (DAC units)'][2:7])

## Old Prepulse creator

In [135]:
class prepulse_creator: 
    def __init__(self, config_file, storage_man_file):
        '''
        Creates pulses of the form 
        # [[frequency], [gain], [length (us)], [phases], [drive channel], [shape], [ramp sigma]], drive channel=1 (flux low), 2 (qubit),3 (flux high),6 (storage),5 (f0g1),4 (manipulate),           
        '''
        # config 
        with open(config_file, 'r') as cfg_file:
            yaml_cfg = yaml.safe_load(cfg_file)
        self.cfg = AttrDict(yaml_cfg)

        # man storage swap data 
        self.dataset = storage_man_swap_dataset(storage_man_file)
        
        # initialize pulse 
        self.pulse = np.array([[],[],[],[],[],[],[]], dtype = object)
    
    def append(self, pulse):
        self.pulse = np.concatenate((self.pulse, pulse), axis=1)
        return None
        
    def qubit(self, pulse_name,ge_bool):
        ''' pulse name comes from yaml file '''
        # frequency 
        if ge_bool: 
            freq = self.cfg.device.qubit.f_ge[0]
        else: 
            freq = self.cfg.device.qubit.f_ef[0]
        
        # length and sigma 
        # pulse_type = self.cfg.device.qubit.pulses[pulse_name]['type']
        # if pulse is 'gauss': 
        #     length = 0
        #     ramp_sigma = self.cfg.device.qubit.pulses[pulse_name]['sigma']
        qubit_pulse = np.array([[freq], 
                [self.cfg.device.qubit.pulses[pulse_name]['gain'][0]],
                [0],
                [0],
                [2],
                ['gaussian'],
                [self.cfg.device.qubit.pulses[pulse_name]['sigma'][0]]], dtype = object)
        self.pulse = np.concatenate((self.pulse, qubit_pulse), axis=1)
        return None
    
    def man(self, cav_name, pulse_name, buffer = False):
        '''name can be pi or hpi'''

        if pulse_name == 'pi': 
            length = self.dataset.get_pi(cav_name)
        else:
            length = self.dataset.get_h_pi(cav_name)
        
        f0g1  = np.array([[self.dataset.get_freq(cav_name)],
                [ self.dataset.get_gain(cav_name)],
                [length],
                [0],
                [5], # f0g1 pulse 
                ['flat_top'],
                [0.005]], dtype = object)
        
        self.pulse = np.concatenate((self.pulse, f0g1), axis=1)
        return None
    # def buffer(self): 
    #     buffer = np.array([[0],
    #             [0],
    #             [0.01],
    #             [0],
    #             [5],
    #             ['flat_top'],
    #             [0.005]], dtype = object)
    #     self.pulse = np.concatenate((self.pulse, buffer), axis=1)
    #     return None

    def storage(self, stor_name, pulse_name):
        '''
        plays sideband pulse on storage via coupler rf flux
        name can be pi or hpi'''

        if pulse_name == 'pi': 
            length = self.dataset.get_pi(stor_name)
        else:
            length = self.dataset.get_h_pi(stor_name)
        freq = self.dataset.get_freq(stor_name)
        if freq<1000: 
            ch = 1
        else:
            ch = 3
        
        storage_pulse = np.array([[self.dataset.get_freq(stor_name)],
                [ self.dataset.get_gain(stor_name)],
                [length],
                [0],
                [ch],
                ['flat_top'],
                [0.005]], dtype = object)
        
        self.pulse = np.concatenate((self.pulse, storage_pulse), axis=1)
        return None
        
                
    

In [ ]:
creator = prepulse_creator(config_file, 'man1_storage_swap_dataset.csv')
modes = ['M1-S2', 'M1-S3', 'M1-S4', 'M1-S5', 'M1-S6', 'M1-S7']
# for mode in modes: 
#     creator.append(np.array([[4984.026194477733],
#                 [3750],
#                 [0],
#                 [0],
#                 [4],
#                 ['gaussian'],
#                 [0.1]], dtype = object))
#     creator.storage(mode, 'pi')
# creator.append(np.array([[4984.026194477733],
#                 [3750],
#                 [0],
#                 [0],
#                 [4],
#                 ['gaussian'],
#                 [0.1]], dtype = object))
# creator.storage('M1-S3', 'pi')
# creator.qubit('pi_ge', ge_bool = True)
# creator.qubit('pi_ef', ge_bool = False)
# # creator.buffer()
# creator.man('M1', 'pi', buffer = True)
# creator.qubit('pi_ef', ge_bool = False)
# creator.storage('M1-S1', 'pi')
# creator.storage('M1-S3', 'pi')
creator.storage('M1-S5', 'pi')
# creator.qubit('pi_ge', ge_bool = True)
# creator.qubit('pi_ef', ge_bool = False)
# creator.man('M2', 'pi')
# creator.pulse
# creator.append(np.array([[4984.026194477733],
#         [3750],
#         [0],
#         [0],
#         [4],
#         ['gaussian'],
#         [0.1]], dtype = object))
# creator.append(np.array([[3568.1904347121535, 3425.5537944060666, 2006.72, 2313.3, 100],
#        [9653, 7575, 15000, 5000, 0],
#        [0, 0, 0.6168960348389007, 10, 5],
#        [0, 0, 0, 0, 0],
#        [2, 2, 5, 3, 3],
#        ['gaussian', 'gaussian', 'flat_top', 'flat_top', 'flat_top'],
#        [0.035, 0.035, 0.005, 0.005, 0.005]], dtype = object))'"LO id is 10003495.txt"
# creator.append(np.array([[3567.900564],
#         [9653],
#         [0],
#         [0],
#         [2],
#         ['gaussian'],
#         [0.035]], dtype = object))
creator.pulse


In [ ]:
creator = prepulse_creator(config_file, 'man1_storage_swap_dataset.csv')
#creator.pulse = np.array([[],[],[],[],[],[],[]], dtype = object)
# creator.qubit('pi_ge', ge_bool = True)
# creator.qubit('pi_ef', ge_bool = False)
# # creator.buffer()
creator.storage('M1-S1', 'pi')
creator.man('M1', 'pi')
creator.pulse

## New Creator

In [24]:
from multimode_expts.MM_base import *
mm_base = MM_base(cfg = yaml_cfg)

man_idx = 1
stor_id = [1,2]
parity_str = [['qubit', 'ge', 'hpi', 0 ], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx), 0 ], 
                ['qubit', 'ge', 'hpi', 0 ], 
                ]

# parity_str_multiple

prep_f0g1_str = [
                ['qubit', 'ge', 'pi', 0],
                ['qubit', 'ef', 'pi', 0],
                ['man', 'M1', 'pi', 0], 
                ['storage', 'M' + str(man_idx) + '-S' + str(stor_id[0]), 'pi', 0]
            ]

    
creator = mm_base.get_prepulse_creator(prep_f0g1_str)
creator.pulse
# pulse_str

array([[3568.1751638611518, 3425.5564066067, 2007.06, 349.415],
       [10716, 8236, 15000, 3000],
       [0, 0, 0.651287, 0.9771428571428572],
       [0, 0, 0, 0],
       [2, 2, 0, 1],
       ['g', 'g', 'flat_top', 'flat_top'],
       [0.035, 0.035, 0.005, 0.005]], dtype=object)

In [ ]:
# for the cross storage phase correction 
spectator_mode_no = 2
target_mode_no = 1
man_no = 1

cfg = yaml_cfg

omega_target = cfg.device.storage.idling_freq[target_mode_no -1]
omega_f0g1 = cfg.device.manipulate.idling_freq[man_no - 1]
phi_target = cfg.device.storage.idling_phase[target_mode_no -1][target_mode_no -1]
phi_f0g1 = cfg.device.manipulate.idling_phase[man_no - 1][man_no - 1]

mm_base = MM_base(cfg = yaml_cfg)

# create prepulse 
prepulse_str = [['qubit', 'ge', 'hpi',0]]
stor_input = mm_base.compound_storage_gate(input = True, storage_no = target_mode_no)

# spector pulse 
spectator_pulse_str = [['storage', 'M' + str(man_no) + '-S' + str(spectator_mode_no), 'pi', 0]]

# idling_times 
spec_idling_time = mm_base.get_total_time(spectator_pulse_str, gate_based = True)
targ_idling_time = mm_base.get_total_time([['storage', 'M' + str(man_no) + '-S' + str(target_mode_no), 'pi', 0]], gate_based = True)

stor_output_str = mm_base.compound_storage_gate(input = False, storage_no = target_mode_no)
ge_virtual_phase = omega_target * spec_idling_time + omega_f0g1 * targ_idling_time + phi_target + phi_f0g1
post_pulse_str = [['qubit', 'ge', 'hpi', ge_virtual_phase]]

# Concatenate all lists along axis 0
concatenated_list = prepulse_str + stor_input + spectator_pulse_str + stor_output_str + post_pulse_str

# Print the concatenated list
concatenated_list


In [ ]:
concatenated_list = [concatenated_list[-1]]

## Example

In [ ]:
# import sequential_experiment as seq_exp
# experiment_name = 'length_rabi_sweep'

# eval('seq_exp.' + experiment_name)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

In [ ]:
from scipy.special import erf

In [ ]:
from experiments.single_qubit.length_rabi_general import LengthRabiGeneralExperiment

cfg_expt = {'start': 0.0069754464285714285, # us
            'step': 0.007, # us
            'qubits': [0],
            'expts': 1,
            'reps': 100,
            'rounds': 1,
            'gain': 0 ,# qubit gain [DAC units],
            'ramp_sigma': 0.005 , # us
            'freq': 3566.383307324457,   #2010.5265269004717,   # MHz
            'pi_ge_before': False,
            'pi_ef_before': False,
            'pi_ge_after': False,
            'pre_pulse': False}

# g state 
lrabi = LengthRabiGeneralExperiment(
    soccfg=soc,
    path=expt_path,
    config_file=config_path,
)
lrabi.cfg.expt = cfg_expt

#g_data = lrabi.acquire()
lrabi.go(analyze=False, display=False, progress=False, save=False)

# Extrapolate f0g1 sideband

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

# Sample data
x_data = np.asarray([0.593264249,0.64507772,	0.670984456	,0.72279792,	0.800518135,	0.878238342])
y_data = np.array([2012.15, 2010.2,	2008.9,	2005.2,		1995.55,		1976.5
])

# Sort the data
sorted_indices = np.argsort(x_data)
x_data = x_data[sorted_indices]
y_data = y_data[sorted_indices]

# Fit a spline to the data
spline = UnivariateSpline(x_data, y_data)

# Evaluate the spline at a range of x values
x_fit = np.linspace(x_data.min(), x_data.max(), 1000)
y_fit = spline(x_fit)

# Plot the original data and the fitted spline
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_data, 'ro', label='Original Data')
plt.plot(x_fit, y_fit, 'b-', label='Fitted Spline')
plt.xlabel('DC flux (mA)')
plt.ylabel('F0-g1 frequency (MHz)')
plt.title('Spline Fit to Data')
plt.legend()
plt.show()

In [ ]:
spline((0.601036269
))

In [ ]:
spline(0.95)

In [ ]:
man_f = -1976.5+(3568.1063715816226+3425.0265059169296)
man_coup =247.8
coup = man_f - man_coup
flux_quanta = 0.878238342#0.670984456
Ec = 60
a = (coup + Ec)/np.sqrt(8*Ec* np.abs(np.cos(np.pi*flux_quanta)))
a

In [ ]:
a**2

In [ ]:
coup

In [ ]:
a* sqrt(8*Ec)

In [ ]:
2*np.pi*flux_quanta

In [ ]:
flux_quanta = 0.64507772
coup = 5235 * np.sqrt(np.abs(np.cos(np.pi*flux_quanta)))

In [ ]:
coup

In [ ]:
man_f - 841.05

In [ ]:
man_f - coup

# Extrapolate Coupler Manipulate Sideband


In [ ]:
y_data = np.asarray([1472.6,	1229.4,	841.05,		451.2,		247.8])
x_data = np.asarray([0.64507772,	0.670984456	,0.72279792,	0.800518135,	0.878238342])
# Sort the data
sorted_indices = np.argsort(x_data)
x_data = x_data[sorted_indices]
y_data = y_data[sorted_indices]

# Fit a spline to the data
spline = UnivariateSpline(x_data, y_data)

# Evaluate the spline at a range of x values
x_fit = np.linspace(x_data.min(), x_data.max(), 1000)
y_fit = spline(x_fit)

# Plot the original data and the fitted spline
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_data, 'ro', label='Original Data')
plt.plot(x_fit, y_fit, 'b-', label='Fitted Spline')
plt.xlabel('DC flux (mA)')
plt.ylabel('Coupler Manipulate sideband (MHz)')
plt.title('Spline Fit to Data')
plt.legend()
plt.show()


In [ ]:
spline(0.601036269
)

# Extrapolate Coupler Frequency 

In [ ]:
y_data = np.asarray([3510.332877,	3754.832877,	4146.882877,		4546.382877,		4768.832877])
x_data = np.asarray([0.64507772,	0.670984456	,0.72279792,	0.800518135,	0.878238342])
# Sort the data
sorted_indices = np.argsort(x_data)
x_data = x_data[sorted_indices]
y_data = y_data[sorted_indices]

# Fit a spline to the data
spline = UnivariateSpline(x_data, y_data)

# Evaluate the spline at a range of x values
x_fit = np.linspace(x_data.min(), x_data.max(), 1000)
y_fit = spline(x_fit)

# Plot the original data and the fitted spline
plt.figure(figsize=(10, 6))
plt.plot(x_data, y_data, 'ro', label='Original Data')
plt.plot(x_fit, y_fit, 'b-', label='Fitted Spline')
#plot extrapolated point at x = 1
plt.plot(1, spline(1), 'go', label='Extrapolated Point')
#add a xline at the above point
plt.axhline(y = spline(1), color='g', linestyle='--', label = str(spline(1)))
plt.xlabel('DC flux (mA)')
plt.ylabel('Coupler (MHz)')
plt.title('Spline Fit to Data')
plt.legend()
plt.show()

In [ ]:
spline(0.601036269
)